In [1]:
import cracknuts as cn

In [2]:
cracker = cn.new_cracker('192.168.0.251')

In [3]:
cracker.connect() # 连接设备

In [4]:
cracker.get_operator().connect()

True

In [5]:
cracker.get_id() # 获取设备ID

(0, 'unknown')

In [6]:
cracker.get_hwardware() # 获取设备名称

AttributeError: 'CrackerS1' object has no attribute 'get_name'

In [ ]:
cracker.get_version() # 获取设备版本信息

In [ ]:
import random
import time
from enum import Enum


class AlgType(Enum):  # Algrithom type
    AES_ENC = 1
    AES_DEC = 2
    DES_ENC = 3
    DES_DEC = 4

sample_length = 1024 * 20
alg = AlgType.AES_ENC # 修改这个值改变算法
# alg = AlgType.DES_ENC # 修改这个值改变算法

# AES Command
cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_set_aes_dec_key = "01 01 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"
cmd_aes_dec = "01 03 00 00 00 00 00 10"
aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

# DES Command
cmd_set_des_enc_key = "02 00 00 00 00 00 00 08"
cmd_set_des_dec_key = "02 01 00 00 00 00 00 08"
cmd_des_enc = "02 02 00 00 00 00 00 08"
cmd_des_dec = "02 03 00 00 00 00 00 08"
des_key = "12 34 56 78 90 ab cd ef"
des_data_len = 8


def init(c):
    cracker.nut_voltage_disable()
    cracker.nut_clock_disable()
    time.sleep(1)
    cracker.nut_clock_freq('8M')
    cracker.nut_clock_enable()
    time.sleep(0.1)
    cracker.nut_voltage_mv(3400)
    cracker.nut_voltage_enable()
    cracker.uart_enable()
    cracker.osc_sample_clock_rate('48m')
    cracker.osc_sample_len(sample_length)
    if alg is AlgType.AES_ENC:
        cmd = cmd_set_aes_enc_key + aes_key
    elif alg is AlgType.AES_DEC:
        cmd = cmd_set_aes_dec_key + aes_key
    elif alg is AlgType.DES_ENC:
        cmd = cmd_set_des_enc_key + des_key
    else:
        cmd = cmd_set_des_dec_key + des_key

    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(c):

    if alg is AlgType.AES_ENC:
        cmd = cmd_aes_enc
        data_len = aes_data_len
    elif alg is AlgType.AES_DEC:
        cmd = cmd_aes_dec
        data_len = aes_data_len
    elif alg is AlgType.DES_ENC:
        cmd = cmd_des_enc
        data_len = des_data_len
    else:
        cmd = cmd_des_dec
        data_len = des_data_len

    # convert cmd to bytes
    cmd = cmd.replace(' ', '')
    cmd = bytes.fromhex(cmd)

    if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC:
        data = random.randbytes(aes_data_len)
    else:
        data = random.randbytes(des_data_len)

    d = cmd + data
    # print(d.hex())

    if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC:
        status, ret = cracker.uart_transmit_receive(d, rx_count= 6 + aes_data_len, is_trigger=True)
        return data + ret[-aes_data_len:]
    else:
        status, ret = cracker.uart_transmit_receive(d, rx_count=6 + des_data_len, is_trigger=True)
        return data + ret[-des_data_len:]


acq = cn.new_acquisition(cracker, do=do, init=init, data_length=aes_data_len * 2 if alg is AlgType.AES_ENC else des_data_len * 2)

In [ ]:
cn.panel(acq)

In [ ]:
# 显示曲线分析面板
tap = cn.panel_trace()
tap

In [ ]:
# 引入波形文件数据集对象
from cracknuts.trace.trace import ScarrTraceDataset
# 加载波形文件
scarr_trace_dataset = ScarrTraceDataset.load("dataset/20250213143033.zarr")
# 将数据集配置到波形显示面板
tap.set_trace_dataset(scarr_trace_dataset)
# 显示第二通道的前十条曲线
tap.show_trace[0, :10]